Connected to CHsolvers (Python 3.9.20)

In [ ]:
import numpy as np

# Auxiliary function to create a meshgrid similar to MATLAB's meshgrid


def meshgrid(x, y):
    x = np.asarray(x).reshape(-1, 1)  # column vector
    y = np.asarray(y).reshape(1, -1)  # row vector
    X = np.repeat(x, y.shape[1], axis=1)
    Y = np.repeat(y, x.shape[0], axis=0)
    return X, Y


def compute_kx(nx, Lx):
    k = np.concatenate((
        np.arange(0, nx // 2 + 1),
        np.arange(-nx // 2 + 1, 0)
    ))
    kx = 1j * k * (2 * np.pi / Lx)
    return kx


def ext(x):

    [nx, ny] = x.shape
    x_ext = np.zeros((2 * nx, 2 * ny))

    # Original block
    x_ext[0:nx, 0:ny] = x

    # Flip horizontally
    x_ext[0:nx, ny:2*ny] = x[:, ::-1]

    # Flip vertically
    x_ext[nx:2*nx, 0:ny] = x[::-1, :]

    # Flip both
    x_ext[nx:2*nx, ny:2*ny] = x[::-1, ::-1]
    return x_ext


def f(phi):
    # f = @(x) 0.25*(x.^2-1).^2
    fphi = (phi ** 2 - 1) ** 2 / 4
    return fphi


def r0_fun(phi0, hx, hy, C0):
    fphi = f(phi0)
    r0 = np.sqrt(hx * hy * np.sum(fphi) + C0)
    return r0


def extback(x_ext):
    # Shrinks from 2*nx x 2*ny back to nx x ny (upper-left block)
    [nx_ext, ny_ext] = (x_ext).shape
    nx = int(nx_ext / 2)
    ny = int(ny_ext / 2)
    x_back = x_ext[0:nx, 0:ny]

    return x_back


def df(phi, gamma0):
    return phi ** 3 - (1 + gamma0) * phi


def Lap_SAV(phi, k2):
    phi_hat = fft_filtered(phi)
    result = np.real(np.fft.ifft(k2 * phi_hat))
    return result


def A_inv_CN(phi, dt, k2, k4, gamma0, epsilon2):
    denom = 1 + (dt / 2) * epsilon2 * k4 - (dt / 2) * gamma0 * k2
    return np.real(np.fft.ifft(fft_filtered(phi) / denom))


def fft_filtered(x):  # removed real(), good
    return (np.fft.fft(x))


def b_fun(phi, hx, hy, C0, gamma0):
    e1 = fft_filtered(f(phi))
    return df(phi, gamma0) / np.sqrt(e1[1, 1] * hx * hy + C0)


def g_fun_CN(phi0, r0, b, dt, hx, hy, epsilon2, gamma0, Beta, C0, k2):
    Lap_phi0 = Lap_SAV(phi0, k2)
    Lap_Lap_phi0 = Lap_SAV(Lap_phi0, k2)

    bphi0 = fft_filtered(b * phi0)
    bphi0 = hx * hy * bphi0[1, 1]

    e1 = fft_filtered(f(phi0))

    g = phi0 - (dt / 2) * epsilon2 * Lap_Lap_phi0 + (dt / 2) * gamma0 * Lap_phi0 + dt * Lap_SAV(b, k2) * \
        (r0 - (1 / 4) * bphi0 - (1 / 2) * Beta * dt *
         r0 * (r0 - np.sqrt(e1[1, 1] * hx * hy + C0)))

    return g


def r_fun(phi, phi0, r0, b, hx, hy, C0, Beta, dt):
    bphi0 = fft_filtered(b * phi0)
    bphi0 = hx * hy * bphi0[1, 1]

    bphi = fft_filtered(b * phi)
    bphi = hx * hy * bphi[1, 1]

    e1 = fft_filtered(f(phi0))

    r = r0 + (1 / 2) * (bphi - bphi0) - Beta * dt * \
        r0 * (r0 - np.sqrt(e1[1, 1] * hx * hy + C0))

    return r

In [ ]:
x = np.array([[1,2],[3,4]])
xext = ext(x)

In [ ]:
xext

array([[1., 2., 2., 1.],
       [3., 4., 4., 3.],
       [3., 4., 4., 3.],
       [1., 2., 2., 1.]])

In [ ]:
f(x)

array([[ 0.  ,  2.25],
       [16.  , 56.25]])

In [ ]:
f(xext)

array([[ 0.  ,  2.25,  2.25,  0.  ],
       [16.  , 56.25, 56.25, 16.  ],
       [16.  , 56.25, 56.25, 16.  ],
       [ 0.  ,  2.25,  2.25,  0.  ]])

In [ ]:
[nx, ny] = x.shape

In [ ]:
domain=[1, 0, 1, 0]

In [ ]:
boundary = 'neumann'
xright, xleft, yright, yleft = domain
Lx = xright - xleft
Ly = yright - yleft

# % Decide on the solver's mesh spacing for NEUMANN vs PERIODIC
# %  - For Neumann: we will mirror the domain, so pass 2*hx and 2*hy into sav_solver.
# %  - For Periodic: keep as-is.
if boundary == "neumann":
    Lx = 2 * Lx
    Ly = 2 * Ly
    nx = 2 * nx
    ny = 2 * ny
hx = Lx / nx
hy = Ly / ny
h2 = hx * hy  # Define mesh size

In [ ]:
epsilon2=np.nan

In [ ]:
m = 4

if np.isnan(epsilon2):
    # Define ϵ^2 if not prespecified
    epsilon2 = h2 * m ** 2 / (2 * np.sqrt(2) * np.arctanh(0.9)) ** 2
else:
    # Else overwrite m
    m = np.sqrt((epsilon2 * (2 * np.sqrt(2) * np.arctanh(0.9)) ** 2) / h2)


In [ ]:
epsilon2

np.float64(0.23068793362547763)

In [ ]:
h2

0.25

In [ ]:

    kx = compute_kx(nx, Lx)
    ky = compute_kx(ny, Ly)

    kxx = kx**2
    kyy = ky**2
    kxx_mat, kyy_mat = meshgrid(kxx, kyy)

    k2 = kxx_mat + kyy_mat
    k4 = k2**2

In [ ]:
k4

array([[   0.        +0.j,   97.40909103-0.j, 1558.54545654-0.j,
          97.40909103-0.j],
       [  97.40909103-0.j,  389.63636414-0.j, 2435.22727585-0.j,
         389.63636414-0.j],
       [1558.54545654-0.j, 2435.22727585-0.j, 6234.18182618-0.j,
        2435.22727585-0.j],
       [  97.40909103-0.j,  389.63636414-0.j, 2435.22727585-0.j,
         389.63636414+0.j]])

In [ ]:
#k4 matches julia

In [ ]:
C0 = 0
Beta = 0
gamma0 = 0

In [ ]:
r0_fun(xext,hx,hy,C0)

np.float64(8.631338250816034)

In [ ]:
df(xext, gamma0)

array([[ 0.,  6.,  6.,  0.],
       [24., 60., 60., 24.],
       [24., 60., 60., 24.],
       [ 0.,  6.,  6.,  0.]])

In [ ]:
Lap_SAV(xext, k2)

array([[   4.9348022 ,   -4.9348022 ,   -4.9348022 ,    4.9348022 ],
       [ -24.674011  ,  -44.4132198 ,  -44.4132198 ,  -24.674011  ],
       [-113.50045061, -162.84847262, -162.84847262, -113.50045061],
       [  -4.9348022 ,  -24.674011  ,  -24.674011  ,   -4.9348022 ]])

In [ ]:
k2

array([[  0.        +0.j,  -9.8696044 +0.j, -39.4784176 +0.j,
         -9.8696044 +0.j],
       [ -9.8696044 +0.j, -19.7392088 +0.j, -49.34802201+0.j,
        -19.7392088 +0.j],
       [-39.4784176 +0.j, -49.34802201+0.j, -78.95683521+0.j,
        -49.34802201+0.j],
       [ -9.8696044 +0.j, -19.7392088 +0.j, -49.34802201+0.j,
        -19.7392088 -0.j]])

In [ ]:
fft_filtered(xext)


array([[ 6.+0.j, -1.-1.j,  0.+0.j, -1.+1.j],
       [14.+0.j, -1.-1.j,  0.+0.j, -1.+1.j],
       [14.+0.j, -1.-1.j,  0.+0.j, -1.+1.j],
       [ 6.+0.j, -1.-1.j,  0.+0.j, -1.+1.j]])

In [ ]:
np.fft.fft(xext, axis = 0)

array([[ 8.+0.j, 12.+0.j, 12.+0.j,  8.+0.j],
       [-2.-2.j, -2.-2.j, -2.-2.j, -2.-2.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [-2.+2.j, -2.+2.j, -2.+2.j, -2.+2.j]])

In [ ]:
np.fft.fft(xext, axis = 0, norm = "forward")

array([[ 2. +0.j ,  3. +0.j ,  3. +0.j ,  2. +0.j ],
       [-0.5-0.5j, -0.5-0.5j, -0.5-0.5j, -0.5-0.5j],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0.5+0.5j, -0.5+0.5j, -0.5+0.5j, -0.5+0.5j]])

In [ ]:
np.fft.fft2(xext)

array([[40.+0.j, -4.-4.j,  0.+0.j, -4.+4.j],
       [-8.-8.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [-8.+8.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [ ]:
import numpy as np

# Auxiliary function to create a meshgrid similar to MATLAB's meshgrid


def meshgrid(x, y):  # good
    x = np.asarray(x).reshape(-1, 1)  # column vector
    y = np.asarray(y).reshape(1, -1)  # row vector
    X = np.repeat(x, y.shape[1], axis=1)
    Y = np.repeat(y, x.shape[0], axis=0)
    return X, Y


def compute_kx(nx, Lx):  # good
    k = np.concatenate((
        np.arange(0, nx // 2 + 1),
        np.arange(-nx // 2 + 1, 0)
    ))
    kx = 1j * k * (2 * np.pi / Lx)
    return kx


def ext(x):  # good

    [nx, ny] = x.shape
    x_ext = np.zeros((2 * nx, 2 * ny))

    # Original block
    x_ext[0:nx, 0:ny] = x

    # Flip horizontally
    x_ext[0:nx, ny:2*ny] = x[:, ::-1]

    # Flip vertically
    x_ext[nx:2*nx, 0:ny] = x[::-1, :]

    # Flip both
    x_ext[nx:2*nx, ny:2*ny] = x[::-1, ::-1]
    return x_ext


def f(phi):  # good
    # f = @(x) 0.25*(x.^2-1).^2
    fphi = (phi ** 2 - 1) ** 2 / 4
    return fphi


def r0_fun(phi0, hx, hy, C0):  # good
    fphi = f(phi0)
    r0 = np.sqrt(hx * hy * np.sum(fphi) + C0)
    return r0


def extback(x_ext):  # good
    # Shrinks from 2*nx x 2*ny back to nx x ny (upper-left block)
    [nx_ext, ny_ext] = (x_ext).shape
    nx = int(nx_ext / 2)
    ny = int(ny_ext / 2)
    x_back = x_ext[0:nx, 0:ny]

    return x_back


def df(phi, gamma0):  # good
    return phi ** 3 - (1 + gamma0) * phi


def Lap_SAV(phi, k2):
    phi_hat = fft_filtered(phi)
    result = np.real(np.fft.ifft2(k2 * phi_hat))
    return result


def A_inv_CN(phi, dt, k2, k4, gamma0, epsilon2):
    denom = 1 + (dt / 2) * epsilon2 * k4 - (dt / 2) * gamma0 * k2
    return np.real(np.fft.ifft2(fft_filtered(phi) / denom))


def fft_filtered(x):  # removed real(), good
    return (np.fft.fft2(x))


def b_fun(phi, hx, hy, C0, gamma0):
    e1 = fft_filtered(f(phi))
    return df(phi, gamma0) / np.sqrt(e1[1, 1] * hx * hy + C0)


def g_fun_CN(phi0, r0, b, dt, hx, hy, epsilon2, gamma0, Beta, C0, k2):
    Lap_phi0 = Lap_SAV(phi0, k2)
    Lap_Lap_phi0 = Lap_SAV(Lap_phi0, k2)

    bphi0 = fft_filtered(b * phi0)
    bphi0 = hx * hy * bphi0[1, 1]

    e1 = fft_filtered(f(phi0))

    g = phi0 - (dt / 2) * epsilon2 * Lap_Lap_phi0 + (dt / 2) * gamma0 * Lap_phi0 + dt * Lap_SAV(b, k2) * \
        (r0 - (1 / 4) * bphi0 - (1 / 2) * Beta * dt *
         r0 * (r0 - np.sqrt(e1[1, 1] * hx * hy + C0)))

    return g


def r_fun(phi, phi0, r0, b, hx, hy, C0, Beta, dt):
    bphi0 = fft_filtered(b * phi0)
    bphi0 = hx * hy * bphi0[1, 1]

    bphi = fft_filtered(b * phi)
    bphi = hx * hy * bphi[1, 1]

    e1 = fft_filtered(f(phi0))

    r = r0 + (1 / 2) * (bphi - bphi0) - Beta * dt * \
        r0 * (r0 - np.sqrt(e1[1, 1] * hx * hy + C0))

    return r

In [ ]:
Lap_SAV(xext, k2)

array([[ 14.8044066,   4.9348022,   4.9348022,  14.8044066],
       [ -4.9348022, -14.8044066, -14.8044066,  -4.9348022],
       [ -4.9348022, -14.8044066, -14.8044066,  -4.9348022],
       [ 14.8044066,   4.9348022,   4.9348022,  14.8044066]])

In [ ]:
A_inv_CN(xext, 1e-5, k2, k4, gamma0, epsilon2)

array([[1.00016851, 2.00005617, 2.00005617, 1.00016851],
       [2.99994383, 3.99983149, 3.99983149, 2.99994383],
       [2.99994383, 3.99983149, 3.99983149, 2.99994383],
       [1.00016851, 2.00005617, 2.00005617, 1.00016851]])

In [ ]:
b_fun(xext, hx, hy, C0, gamma0)

array([[0.        +0.j        , 0.97332853-0.97332853j,
        0.97332853-0.97332853j, 0.        +0.j        ],
       [3.89331411-3.89331411j, 9.73328527-9.73328527j,
        9.73328527-9.73328527j, 3.89331411-3.89331411j],
       [3.89331411-3.89331411j, 9.73328527-9.73328527j,
        9.73328527-9.73328527j, 3.89331411-3.89331411j],
       [0.        +0.j        , 0.97332853-0.97332853j,
        0.97332853-0.97332853j, 0.        +0.j        ]])

In [ ]:
 e1 = fft_filtered(f(xext))
 e1

array([[ 298.  +0.j,  -85. -85.j,    0.  +0.j,  -85. +85.j],
       [-140.-140.j,    0. +76.j,    0.  +0.j,   76.  +0.j],
       [   0.  +0.j,    0.  +0.j,    0.  +0.j,    0.  +0.j],
       [-140.+140.j,   76.  +0.j,    0.  +0.j,    0. -76.j]])

In [ ]:
np.sqrt(e1[1, 1])

np.complex128(6.164414002968976+6.164414002968977j)

In [ ]:
np.sqrt(e1[0, 0])

np.complex128(17.26267650163207+0j)

In [ ]:
import numpy as np

# Auxiliary function to create a meshgrid similar to MATLAB's meshgrid


def meshgrid(x, y):  # good
    x = np.asarray(x).reshape(-1, 1)  # column vector
    y = np.asarray(y).reshape(1, -1)  # row vector
    X = np.repeat(x, y.shape[1], axis=1)
    Y = np.repeat(y, x.shape[0], axis=0)
    return X, Y


def compute_kx(nx, Lx):  # good
    k = np.concatenate((
        np.arange(0, nx // 2 + 1),
        np.arange(-nx // 2 + 1, 0)
    ))
    kx = 1j * k * (2 * np.pi / Lx)
    return kx


def ext(x):  # good

    [nx, ny] = x.shape
    x_ext = np.zeros((2 * nx, 2 * ny))

    # Original block
    x_ext[0:nx, 0:ny] = x

    # Flip horizontally
    x_ext[0:nx, ny:2*ny] = x[:, ::-1]

    # Flip vertically
    x_ext[nx:2*nx, 0:ny] = x[::-1, :]

    # Flip both
    x_ext[nx:2*nx, ny:2*ny] = x[::-1, ::-1]
    return x_ext


def f(phi):  # good
    # f = @(x) 0.25*(x.^2-1).^2
    fphi = (phi ** 2 - 1) ** 2 / 4
    return fphi


def r0_fun(phi0, hx, hy, C0):  # good
    fphi = f(phi0)
    r0 = np.sqrt(hx * hy * np.sum(fphi) + C0)
    return r0


def extback(x_ext):  # good
    # Shrinks from 2*nx x 2*ny back to nx x ny (upper-left block)
    [nx_ext, ny_ext] = (x_ext).shape
    nx = int(nx_ext / 2)
    ny = int(ny_ext / 2)
    x_back = x_ext[0:nx, 0:ny]

    return x_back


def df(phi, gamma0):  # good
    return phi ** 3 - (1 + gamma0) * phi


def Lap_SAV(phi, k2):  # good
    phi_hat = fft_filtered(phi)
    result = np.real(np.fft.ifft2(k2 * phi_hat))
    return result


def A_inv_CN(phi, dt, k2, k4, gamma0, epsilon2):  # good
    denom = 1 + (dt / 2) * epsilon2 * k4 - (dt / 2) * gamma0 * k2
    return np.real(np.fft.ifft2(fft_filtered(phi) / denom))


def fft_filtered(x):  # fft2 is equivalent to fft in julia for 2d array, good
    return (np.fft.fft2(x))


def b_fun(phi, hx, hy, C0, gamma0):
    e1 = fft_filtered(f(phi))
    return df(phi, gamma0) / np.sqrt(e1[0, 0] * hx * hy + C0)


def g_fun_CN(phi0, r0, b, dt, hx, hy, epsilon2, gamma0, Beta, C0, k2):
    Lap_phi0 = Lap_SAV(phi0, k2)
    Lap_Lap_phi0 = Lap_SAV(Lap_phi0, k2)

    bphi0 = fft_filtered(b * phi0)
    bphi0 = hx * hy * bphi0[0, 0]

    e1 = fft_filtered(f(phi0))

    g = phi0 - (dt / 2) * epsilon2 * Lap_Lap_phi0 + (dt / 2) * gamma0 * Lap_phi0 + dt * Lap_SAV(b, k2) * \
        (r0 - (1 / 4) * bphi0 - (1 / 2) * Beta * dt *
         r0 * (r0 - np.sqrt(e1[0, 0] * hx * hy + C0)))

    return g


def r_fun(phi, phi0, r0, b, hx, hy, C0, Beta, dt):
    bphi0 = fft_filtered(b * phi0)
    bphi0 = hx * hy * bphi0[0, 0]

    bphi = fft_filtered(b * phi)
    bphi = hx * hy * bphi[0, 0]

    e1 = fft_filtered(f(phi0))

    r = r0 + (1 / 2) * (bphi - bphi0) - Beta * dt * \
        r0 * (r0 - np.sqrt(e1[0, 0] * hx * hy + C0))

    return r

In [ ]:
b_fun(xext, hx, hy, C0, gamma0)

array([[0.        +0.j, 0.69514134+0.j, 0.69514134+0.j, 0.        +0.j],
       [2.78056534+0.j, 6.95141336+0.j, 6.95141336+0.j, 2.78056534+0.j],
       [2.78056534+0.j, 6.95141336+0.j, 6.95141336+0.j, 2.78056534+0.j],
       [0.        +0.j, 0.69514134+0.j, 0.69514134+0.j, 0.        +0.j]])

In [ ]:
dt = 1e-5
r0 = r0_fun(xext, hx, hy, C0) # Initialize sav state

phi0_df   = df(xext,gamma0) #df at phi0
Lap_dfphi0 = Lap_SAV(phi0_df, k2)    #Lap of df(phi0)
phi_bar = A_inv_CN(xext + dt/2 * Lap_dfphi0, dt, k2, k4, gamma0, epsilon2)
b = b_fun(phi_bar,hx,hy,C0,gamma0)

g_fun_CN(xext, r0, b, dt, hx, hy, epsilon2, gamma0, Beta, C0, k2)

array([[1.00003977+0.j, 1.99985086+0.j, 1.99985086+0.j, 1.00003977+0.j],
       [2.99989287+0.j, 4.00021649+0.j, 4.00021649+0.j, 2.99989287+0.j],
       [2.99989287+0.j, 4.00021649+0.j, 4.00021649+0.j, 2.99989287+0.j],
       [1.00003977+0.j, 1.99985086+0.j, 1.99985086+0.j, 1.00003977+0.j]])

In [ ]:
AiLb = A_inv_CN(Lap_SAV(b, k2), dt, k2, k4, gamma0, epsilon2)
Aig = A_inv_CN(g, dt, k2, k4, gamma0, epsilon2)

gamma = -np.real(np.fft.fft2(b*AiLb))
gamma = gamma[0, 0]*hx*hy

# Step 2
bphi = np.real(np.fft.fft2(b*Aig))
bphi = bphi[0, 0]*hx*hy/(1+dt/4*gamma)

# Step 3
phi_new = dt/4*bphi*AiLb + Aig
r_fun(phi, xext, r0, b, hx, hy, C0, Beta, dt)

NameError: name 'g' is not defined

In [ ]:
dt = 1e-5
r0 = r0_fun(xext, hx, hy, C0) # Initialize sav state

phi0_df   = df(xext,gamma0) #df at phi0
Lap_dfphi0 = Lap_SAV(phi0_df, k2)    #Lap of df(phi0)
phi_bar = A_inv_CN(xext + dt/2 * Lap_dfphi0, dt, k2, k4, gamma0, epsilon2)
b = b_fun(phi_bar,hx,hy,C0,gamma0)

g = g_fun_CN(xext, r0, b, dt, hx, hy, epsilon2, gamma0, Beta, C0, k2)

In [ ]:
AiLb = A_inv_CN(Lap_SAV(b, k2), dt, k2, k4, gamma0, epsilon2)
Aig = A_inv_CN(g, dt, k2, k4, gamma0, epsilon2)

gamma = -np.real(np.fft.fft2(b*AiLb))
gamma = gamma[0, 0]*hx*hy

# Step 2
bphi = np.real(np.fft.fft2(b*Aig))
bphi = bphi[0, 0]*hx*hy/(1+dt/4*gamma)

# Step 3
phi_new = dt/4*bphi*AiLb + Aig
r_fun(phi_new, xext, r0, b, hx, hy, C0, Beta, dt)

np.complex128(8.61631321203629+0j)